In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **2. DATASET PROCESSING**

In [3]:
import os
os.makedirs('/content/drive/MyDrive/210411100054-SitiNurAini/DATASET-CXR-AL3/ImageSets/Main', exist_ok=True)

## **MODE 0 [9:1]**



In [8]:
import os
import random
import xml.etree.ElementTree as ET
import numpy as np

# Define get_classes function if utils module is missing
def get_classes(classes_path):
    with open(classes_path, 'r') as f:
        classes = [line.strip() for line in f.readlines()]
    return classes, len(classes)

annotation_mode     = 0
classes_path        = '/content/drive/MyDrive/210411100054-SitiNurAini/model_data/cxr_classes.txt'
trainval_percent    = 0.9
train_percent       = 0.9
CXR_AL3_path        = '/content/drive/MyDrive/210411100054-SitiNurAini/DATASET-CXR-AL3'

CXR_AL3_sets        = [('2024', 'train'), ('2024', 'val')]
classes, _          = get_classes(classes_path)

photo_nums  = np.zeros(len(CXR_AL3_sets))
nums        = np.zeros(len(classes))

# Fungsi untuk mengonversi anotasi
def convert_annotation(year, image_id, list_file):
    in_file = open(os.path.join(CXR_AL3_path, 'annotations', f'{image_id}.xml'), encoding='utf-8')
    tree = ET.parse(in_file)
    root = tree.getroot()

    for obj in root.iter('object'):
        difficult = 0
        if obj.find('difficult') is not None:
            difficult = obj.find('difficult').text
        cls = obj.find('name').text
        if cls not in classes or int(difficult) == 1:
            continue
        cls_id = classes.index(cls)
        xmlbox = obj.find('bndbox')
        b = (int(float(xmlbox.find('xmin').text)), int(float(xmlbox.find('ymin').text)),
             int(float(xmlbox.find('xmax').text)), int(float(xmlbox.find('ymax').text)))
        list_file.write(" " + ",".join([str(a) for a in b]) + ',' + str(cls_id))

        nums[classes.index(cls)] += 1

if __name__ == "__main__":
    random.seed(0)

    if annotation_mode == 0 or annotation_mode == 1:
        print("Generate txt in ImageSets.")
        xmlfilepath = os.path.join(CXR_AL3_path, 'annotations')
        saveBasePath = os.path.join(CXR_AL3_path, 'ImageSets/Main')
        total_xml = [xml for xml in os.listdir(xmlfilepath) if xml.endswith(".xml")]

        num = len(total_xml)
        list_indices = range(num)
        tv = int(num * trainval_percent)
        tr = int(tv * train_percent)
        trainval = random.sample(list_indices, tv)
        train = random.sample(trainval, tr)

        print("train and val size", tv)
        print("train size", tr)
        ftrainval = open(os.path.join(saveBasePath, 'trainval_9:1.txt'), 'w')
        ftest = open(os.path.join(saveBasePath, 'test_9:1.txt'), 'w')
        ftrain = open(os.path.join(saveBasePath, 'train_9:1.txt'), 'w')
        fval = open(os.path.join(saveBasePath, 'val_9:1.txt'), 'w')

        for i in list_indices:
            name = total_xml[i][:-4] + '\n'
            if i in trainval:
                ftrainval.write(name)
                if i in train:
                    ftrain.write(name)
                else:
                    fval.write(name)
            else:
                ftest.write(name)

        ftrainval.close()
        ftrain.close()
        fval.close()
        ftest.close()
        print("Generate txt in ImageSets done.")


    if annotation_mode == 0 or annotation_mode == 2:
        print("Generate 2024_train_9:1.txt and 2024_val_9:1.txt for train.")
        saveBasePath = os.path.join(CXR_AL3_path, 'ImageSets/Main')

        type_index = 0
        for year, image_set in CXR_AL3_sets:
            input_file_path = os.path.join(saveBasePath, f'{image_set}_9:1.txt')
            image_ids = open(input_file_path, encoding='utf-8').read().strip().split()

            output_file_path = os.path.join(saveBasePath, f'{year}_{image_set}_9:1.txt')
            list_file = open(output_file_path, 'w', encoding='utf-8')

            for image_id in image_ids:
                list_file.write(f'{os.path.abspath(CXR_AL3_path)}/JPGImages/{image_id}.jpg')
                convert_annotation(year, image_id, list_file)
                list_file.write('\n')

            photo_nums[type_index] = len(image_ids)
            type_index += 1
            list_file.close()

        print("Generate 2024_train_9:1.txt and 2024_val_9:1.txt for train done.")

        def printTable(List1, List2):
            for i in range(len(List1[0])):
                print("|", end=' ')
                for j in range(len(List1)):
                    print(List1[j][i].rjust(int(List2[j])), end=' ')
                    print("|", end=' ')
                print()

        str_nums = [str(int(x)) for x in nums]
        tableData = [
            classes, str_nums
        ]
        colWidths = [max(len(item) for item in col) for col in tableData]
        printTable(tableData, colWidths)


Generate txt in ImageSets.
train and val size 1027
train size 924
Generate txt in ImageSets done.
Generate 2024_train_9:1.txt and 2024_val_9:1.txt for train.
Generate 2024_train_9:1.txt and 2024_val_9:1.txt for train done.
| Cardiomegaly | 357 | 
|  Nodule_Mass | 360 | 
| Pneumothorax | 336 | 


menampilkan isi train.txt dan val.txt [9:1]

In [12]:
# Fungsi untuk membaca dan menampilkan isi file
def display_file_content(file_path, label, max_lines=5):
    try:
        with open(file_path, 'r') as file:
            print(f"Isi file {label}:")
            print(''.join(file.readlines()[:max_lines]).strip())  # Gabung semua baris yang dipilih
    except FileNotFoundError:
        print(f"File {label} tidak ditemukan.")
    except Exception as e:
        print(f"Error pada file {label}: {e}")

# Base path untuk file
base_path = '/content/drive/MyDrive/210411100054-SitiNurAini/DATASET-CXR-AL3/ImageSets/Main'

# Membaca dan menampilkan file
for label in ['2024_train_9:1.txt', '2024_val_9:1.txt']:
    display_file_content(f"{base_path}/{label}", label)


Isi file 2024_train_9:1.txt:
/content/drive/MyDrive/210411100054-SitiNurAini/DATASET-CXR-AL3/JPGImages/d4a67043b09bca469a46c16b763c0391.jpg 168,235,412,304,0
/content/drive/MyDrive/210411100054-SitiNurAini/DATASET-CXR-AL3/JPGImages/f6349590bcca0313c865a6658b41cde4.jpg 182,292,442,392,0
/content/drive/MyDrive/210411100054-SitiNurAini/DATASET-CXR-AL3/JPGImages/1ed4cf45940bbfbcdc31cf4289c1c1e4.jpg 155,261,360,320,0
/content/drive/MyDrive/210411100054-SitiNurAini/DATASET-CXR-AL3/JPGImages/5f4eff3b76151906fe87be784d191a1f.jpg 154,282,358,322,0
/content/drive/MyDrive/210411100054-SitiNurAini/DATASET-CXR-AL3/JPGImages/281070333d103fe4a02065d7f986b062.jpg 184,276,439,338,0
Isi file 2024_val_9:1.txt:
/content/drive/MyDrive/210411100054-SitiNurAini/DATASET-CXR-AL3/JPGImages/221d68705e17ec11935e410fcc483b0f.jpg 130,209,452,334,0
/content/drive/MyDrive/210411100054-SitiNurAini/DATASET-CXR-AL3/JPGImages/cdeda235172891a9886537799cd57d76.jpg 186,284,420,371,0
/content/drive/MyDrive/210411100054-SitiN

##**MODE 0 [8:2]**


In [9]:
import os
import random
import xml.etree.ElementTree as ET
import numpy as np

# Define get_classes function if utils module is missing
def get_classes(classes_path):
    with open(classes_path, 'r') as f:
        classes = [line.strip() for line in f.readlines()]
    return classes, len(classes)

annotation_mode     = 0
classes_path        = '/content/drive/MyDrive/210411100054-SitiNurAini/model_data/cxr_classes.txt'
trainval_percent    = 0.8
train_percent       = 0.9
CXR_AL3_path        = '/content/drive/MyDrive/210411100054-SitiNurAini/DATASET-CXR-AL3'

CXR_AL3_sets        = [('2024', 'train'), ('2024', 'val')]
classes, _          = get_classes(classes_path)

photo_nums  = np.zeros(len(CXR_AL3_sets))
nums        = np.zeros(len(classes))

# Fungsi untuk mengonversi anotasi
def convert_annotation(year, image_id, list_file):
    in_file = open(os.path.join(CXR_AL3_path, 'annotations', f'{image_id}.xml'), encoding='utf-8')
    tree = ET.parse(in_file)
    root = tree.getroot()

    for obj in root.iter('object'):
        difficult = 0
        if obj.find('difficult') is not None:
            difficult = obj.find('difficult').text
        cls = obj.find('name').text
        if cls not in classes or int(difficult) == 1:
            continue
        cls_id = classes.index(cls)
        xmlbox = obj.find('bndbox')
        b = (int(float(xmlbox.find('xmin').text)), int(float(xmlbox.find('ymin').text)),
             int(float(xmlbox.find('xmax').text)), int(float(xmlbox.find('ymax').text)))
        list_file.write(" " + ",".join([str(a) for a in b]) + ',' + str(cls_id))

        nums[classes.index(cls)] += 1

if __name__ == "__main__":
    random.seed(0)

    if annotation_mode == 0 or annotation_mode == 1:
        print("Generate txt in ImageSets.")
        xmlfilepath = os.path.join(CXR_AL3_path, 'annotations')
        saveBasePath = os.path.join(CXR_AL3_path, 'ImageSets/Main')
        total_xml = [xml for xml in os.listdir(xmlfilepath) if xml.endswith(".xml")]

        num = len(total_xml)
        list_indices = range(num)
        tv = int(num * trainval_percent)
        tr = int(tv * train_percent)
        trainval = random.sample(list_indices, tv)
        train = random.sample(trainval, tr)

        print("train and val size", tv)
        print("train size", tr)
        ftrainval = open(os.path.join(saveBasePath, 'trainval_8:2.txt'), 'w')
        ftest = open(os.path.join(saveBasePath, 'test_8:2.txt'), 'w')
        ftrain = open(os.path.join(saveBasePath, 'train_8:2.txt'), 'w')
        fval = open(os.path.join(saveBasePath, 'val_8:2.txt'), 'w')

        for i in list_indices:
            name = total_xml[i][:-4] + '\n'
            if i in trainval:
                ftrainval.write(name)
                if i in train:
                    ftrain.write(name)
                else:
                    fval.write(name)
            else:
                ftest.write(name)

        ftrainval.close()
        ftrain.close()
        fval.close()
        ftest.close()
        print("Generate txt in ImageSets done.")


    if annotation_mode == 0 or annotation_mode == 2:
        print("Generate 2024_train_8:2.txt and 2024_val_8:2.txt for train.")
        saveBasePath = os.path.join(CXR_AL3_path, 'ImageSets/Main')

        type_index = 0
        for year, image_set in CXR_AL3_sets:
            input_file_path = os.path.join(saveBasePath, f'{image_set}_8:2.txt')
            image_ids = open(input_file_path, encoding='utf-8').read().strip().split()

            output_file_path = os.path.join(saveBasePath, f'{year}_{image_set}_8:2.txt')
            list_file = open(output_file_path, 'w', encoding='utf-8')

            for image_id in image_ids:
                list_file.write(f'{os.path.abspath(CXR_AL3_path)}/JPGImages/{image_id}.jpg')
                convert_annotation(year, image_id, list_file)
                list_file.write('\n')

            photo_nums[type_index] = len(image_ids)
            type_index += 1
            list_file.close()

        print("Generate 2024_train_8:2.txt and 2024_val_8:2.txt for train done.")

        def printTable(List1, List2):
            for i in range(len(List1[0])):
                print("|", end=' ')
                for j in range(len(List1)):
                    print(List1[j][i].rjust(int(List2[j])), end=' ')
                    print("|", end=' ')
                print()

        str_nums = [str(int(x)) for x in nums]
        tableData = [
            classes, str_nums
        ]
        colWidths = [max(len(item) for item in col) for col in tableData]
        printTable(tableData, colWidths)


Generate txt in ImageSets.
train and val size 913
train size 821
Generate txt in ImageSets done.
Generate 2024_train_8:2.txt and 2024_val_8:2.txt for train.
Generate 2024_train_8:2.txt and 2024_val_8:2.txt for train done.
| Cardiomegaly | 325 | 
|  Nodule_Mass | 311 | 
| Pneumothorax | 298 | 


menampilkan isi train.txt dan val.txt [8:2]

In [13]:
# Fungsi untuk membaca dan menampilkan isi file
def display_file_content(file_path, label, max_lines=5):
    try:
        with open(file_path, 'r') as file:
            print(f"Isi file {label}:")
            print(''.join(file.readlines()[:max_lines]).strip())  # Gabung semua baris yang dipilih
    except FileNotFoundError:
        print(f"File {label} tidak ditemukan.")
    except Exception as e:
        print(f"Error pada file {label}: {e}")

# Base path untuk file
base_path = '/content/drive/MyDrive/210411100054-SitiNurAini/DATASET-CXR-AL3/ImageSets/Main'

# Membaca dan menampilkan file
for label in ['2024_train_8:2.txt', '2024_val_8:2.txt']:
    display_file_content(f"{base_path}/{label}", label)


Isi file 2024_train_8:2.txt:
/content/drive/MyDrive/210411100054-SitiNurAini/DATASET-CXR-AL3/JPGImages/d4a67043b09bca469a46c16b763c0391.jpg 168,235,412,304,0
/content/drive/MyDrive/210411100054-SitiNurAini/DATASET-CXR-AL3/JPGImages/f6349590bcca0313c865a6658b41cde4.jpg 182,292,442,392,0
/content/drive/MyDrive/210411100054-SitiNurAini/DATASET-CXR-AL3/JPGImages/1ed4cf45940bbfbcdc31cf4289c1c1e4.jpg 155,261,360,320,0
/content/drive/MyDrive/210411100054-SitiNurAini/DATASET-CXR-AL3/JPGImages/5f4eff3b76151906fe87be784d191a1f.jpg 154,282,358,322,0
/content/drive/MyDrive/210411100054-SitiNurAini/DATASET-CXR-AL3/JPGImages/281070333d103fe4a02065d7f986b062.jpg 184,276,439,338,0
Isi file 2024_val_8:2.txt:
/content/drive/MyDrive/210411100054-SitiNurAini/DATASET-CXR-AL3/JPGImages/5ccbe9cf1b033cccdd76fd99cc6ccd01.jpg 185,286,410,370,0
/content/drive/MyDrive/210411100054-SitiNurAini/DATASET-CXR-AL3/JPGImages/2e02414c9f4b2b23a7761fe8c2b3bc93.jpg 189,302,392,378,0
/content/drive/MyDrive/210411100054-SitiN